# Molecular Dynamics Calculations with Quantum Espresso

When computing nonadiabatic molecular dynamics (NAMD) within the neglect-of-back-reaction approximation (NBRA), one first precomputes a molecular dynamics trajectory.

In this tutorial, we will learn how to compute a molecular dynamics trajectory using the Quantum Espresso software package. Specifically, we use the plane-wave sub-program of the Quantum Espresso package. 

Our example system will be a single TiO2 unit cell (anatase). The dynamics will be computed at the Gamma point, with a plane wave energy cut off of 30 Ry and a charge density cut off of 300 Ry. As such parameters are not converged to any particular property of interest, we warn users that such parameters are chosen only for their extreme computational efficiency, and are most likely too crude to be considered quantitative.

We will be computing just 100 fs of dynamics. That is, 100 steps with a time-step of 1 fs

## Loading the Quantum Espresso Module

In order to use Quantum Espresso, we must first load the module. To do so using the UB CCR, please execute the following command. To do so from the Jupyter Notebook, we must use the "!" mark before the command

In [1]:
!module load espresso/6.2.1

The Intel compilers are in your path. This is adequate for compiling and
running most codes. Source
/util/academic/intel/17.0/compilers_and_libraries_2017/linux/bin/compilervars.sh
for more features. 
 Intel-MPI is in your path. This is adequate for compiling and running most
codes. Source the
/util/academic/intel/17.0/compilers_and_libraries_2017/linux/mpi/intel64/bin/mpivars.sh
file for more features. 
 The MKL Libraries are in your library path. This is adequate for compiling and
running most codes. Source $MKL/bin/mklvars.sh" for more features. 


## Making the Quantum Espresso Input File

Below, will be making the Quantum Espresso input file. Given that our platform is a Jupyter Notebook, we will make the entire input file a single string, and subsequently write this string to our input file. Below, i detail the most important paramters of the input file. For a more complete description, please see the quantum espresso [pw.x input description](https://www.quantum-espresso.org/Doc/INPUT_PW.html). The pseudopotentials used herein can be found by visitin the quantum espresso [pseudopotential table](https://www.quantum-espresso.org/pseudopotentials).

&CONTROL
* `calculation` - 'md', indicates the calculation is a molecular dynamics calculations
* `nstep` - how many timesteps to take in the simulation. Each step is of size dt
* `dt` - size of each simulation step. The step size is in Rydberg atomic units
* `pseudo_dir` - location of the pseudo-potential files

&SYSTEM
* `nat` - total number of atoms in our system (6 in this case)
* `ntype` - how many different types of atoms we have (2 in this case)
* `nspin` - 1 - spin-restricted, 2 - spin-unrestricted, 4 - spin-non-colinear
* `ecutwfc` - value for the plane wave kinetic energy cutoff
* `ecutrho` - value for the charge density cutoff
* `occupations` - how the electrons will be occupying the electron states. 'smearing' - occupations will be smeared over the electornic states
* `smearing` - the type of smearing to be applied on the electronic state occupations. 'gaussian' apply the smearing in a manner consistent with a gaussian distribution centered on the highest occupied state
* `degauss` - parameter related to how the smearing on the electronic state occupations is applied. Higher degauss means more smearing. Analogous to the gaussian uncertainty.

&ELECTRONS
* `electron_maxstep` - max number of SCF cycles to reach convergence
* `conv_thr` - SCF convergence threshold

&IONS
* `ion_dynamics` - algorithm for molecular dynamics
* `ion_temperature` - Thermostat to be used for NVT dynamics
* `tempw` - temperature of the thermostat
* `nraise` - frequency of the thermostat

ATOMIC_SPECIES

The formal here is: 'Element Name' 'atomic mass' 'name of pseudo-potential file'  

K_POINTS

Define the K-point mesh for integration of the Brillouin zone. 'Gamma' selects the Gamma point (center) Another option is to define a mesh. Ex) 2 2 2 0 0 0

CELL_PARAMETERS 

Defines the cell paramters for the system 

x1, y1, z1

x2, y2, z2

x3, y3, x3

ATOMIC_POSITIONS

Defines the atomic position for the system 

Atom label, x,y,z

In [2]:
QE_input_file = \
"""
&CONTROL
  calculation   = 'md',
  nstep         = 100,
  dt            = 20.67055,
  pseudo_dir    = '/util/academic/espresso/6.2.1/pseudopotentials/upf_files',
  outdir        = './',
  prefix        = 'x0',
  disk_io       = 'low',
/

&SYSTEM
  ibrav       = 0,
  nat         = 61,
  ntyp        = 4,
  nspin       = 1,
  ecutwfc     = 30,
  ecutrho     = 300,
  occupations = 'smearing',
  smearing    = 'gaussian',
  degauss     = 0.01,
  nosym       = true,
/

&ELECTRONS
  electron_maxstep = 300,
  conv_thr         = 1.D-4,
  mixing_beta      = 0.30,
/

&IONS
  ion_dynamics      = 'verlet',
  ion_temperature   = 'andersen',
  tempw             = 300,
  nraise            = 10,
  pot_extrapolation = 'second-order',
  wfc_extrapolation = 'second-order',
/

ATOMIC_SPECIES
C  12.010     C.pbe-n-rrkjus_psl.0.1.UPF 
H  1.007      H.pbe-rrkjus_psl.0.1.UPF 
Ir 192.217    Ir.pbe-n-rrkjus_psl.0.2.3.UPF
N  14.006     N.pbe-n-rrkjus_psl.0.1.UPF


K_POINTS gamma

CELL_PARAMETERS  angstrom
       10.394909858703       0.0000000000          0.0000000000
        0.0000000000        10.394909858703        0.0000000000
        0.0000000000         0.0000000000         10.394909858703

ATOMIC_POSITIONS angstrom
Ir         0.00000        0.00000       -0.00413 
N          0.87209       -1.62649       -1.09124 
N         -1.84462        0.05799       -1.09124 
C         -0.59580        1.61665        1.03779 
C          1.69796       -0.29235        1.03779 
C          2.05805       -2.05607       -0.60239 
C          2.53979       -1.32653        0.56161 
N          0.97253        1.56850       -1.09124 
C         -1.10216       -1.32430        1.03779 
C          0.75158        2.81036       -0.60239 
C         -0.12109        2.86279        0.56161 
C         -2.80963       -0.75429       -0.60239 
C         -2.41871       -1.53626        0.56161 
C          2.14660        0.40505        2.16632 
C          3.34657        0.09837        2.78651 
C          4.15815       -0.92316        2.30115 
C          3.75083       -1.63212        1.18984 
C          2.71319       -3.12773       -1.21378 
C          2.15370       -3.74895       -2.30956 
C          0.93234       -3.29720       -2.79244 
C          0.33121       -2.23713       -2.14715 
C          1.77180        1.40540       -2.14715 
C          2.38929        2.45603       -2.79244 
C          2.16983        3.73964       -2.30956 
C          1.35210        3.91356       -1.21378 
C         -0.46196        4.06438        1.18984 
C         -1.27959        4.06265        2.30115 
C         -1.75848        2.84903        2.78651 
C         -1.42409        1.65649        2.16632 
C         -3.28888       -2.43225        1.18984 
C         -2.87856       -3.13949        2.30115 
C         -1.58809       -2.94740        2.78651 
C         -0.72252       -2.06154        2.16632 
C         -2.10301        0.83173       -2.14715 
C         -4.06529       -0.78583       -1.21378 
C         -4.32354        0.00931       -2.30956 
C         -3.32163        0.84117       -2.79244 
H          1.53488        1.20257        2.57233 
H          3.65651        0.66080        3.66181 
H          5.09618       -1.16121        2.78894 
H          4.38255       -2.42935        0.81291 
H          3.65924       -3.47162       -0.81800 
H          2.66048       -4.58242       -2.78245 
H          0.44913       -3.75768       -3.64421 
H         -0.62611       -1.85099       -2.47662 
H          1.91606        0.38327       -2.47662 
H          3.02968        2.26780       -3.64421 
H          2.63826        4.59525       -2.78245 
H          1.17689        4.90481       -0.81800 
H         -0.08739        5.01008        0.81291 
H         -1.54245        4.99402        2.78894 
H         -2.40052        2.83623        3.66181 
H         -1.80889        0.72796        2.57233 
H         -4.29516       -2.58072        0.81291 
H         -3.55372       -3.83281        2.78894 
H         -1.25599       -3.49703        3.66181 
H          0.27402       -1.93053        2.57233 
H         -1.28995        1.46772       -2.47662 
H         -4.83613       -1.43319       -0.81800 
H         -5.29873       -0.01283       -2.78245 
H         -3.47881        1.48988       -3.64421 
"""

## Write the Quantum Espresso Input File

Here, we simply write the Quantum Espresso Input File

In [3]:
#print (QE_input_file)
f = open("x0.md.in","w")
f.write(QE_input_file)
f.close()

## Execute the Quantum Espresso Input File

Our output file will be "x0.md.out". Execution of the following cell may take 1-2 minutes.

In [4]:
!/util/academic/espresso/6.2.1/bin/pw.x < x0.md.in > x0.md.out

## Clean Up the Unneeded Output Files

In [5]:
!sh clean.sh

rm: cannot remove ‘x0.w*’: No such file or directory
rm: cannot remove ‘sl*’: No such file or directory
rm: cannot remove ‘x0.u*’: No such file or directory
rm: cannot remove ‘CR*’: No such file or directory
